In [23]:
import pandas as pd
import numpy as np
from autocorrect import spell # autocorrect information
import spacy # might useless - but is a useful package to implement < - 
from tqdm.auto import tqdm
import helper
from helper import *
import plotly.graph_objects as go
import plotly.express as px
#tqdm.pandas()

In [2]:
input_survey=pd.read_csv('../data/WebsiteSatisfaction.csv')
#start to rename the csv hard coding - throw exception if the required schema in the csv doesnt exist
#drop the na and tell the business to get fucked 
input_survey.dropna(inplace=True)

In [3]:
import importlib
importlib.reload(helper)

<module 'helper' from 'c:\\Users\\Chris Dong\\Desktop\\codeBase\\survey_semantic\\research\\helper.py'>

# data engineering portion

In [4]:
#first implementation of pipe for pandas
input_processed = (input_survey.
pipe(spell_chcker,['Comments']).
pipe(text_blob_polarity_semantic_eng,['Comments_spell_checked']).
pipe(time_stamp_cleaner, 'ActivityCompleted')
)

In [5]:
from classifier import TopicClassifier
tags = ["User Experience", "customer service", "equipment availability", 
"pricing error", 
"incorrect information", 
"login", 
"transaction",
"complexity"]
zero_shot=TopicClassifier(tags) #model init first time is ruff.

In [6]:
# topic classifier
# input_processed["topic"] = input_processed['Comments_spell_checked'].apply(zero_shot.top_prediction)

# visualiation building

In [19]:
# charts and basic information that people might care about ->
nps_best=' '.join(input_survey[input_survey['nps_score'] > 3 ].nlargest(n=50, columns=['Comments_spell_checked_txtblob_polarity','Comments_spell_checked_txtblob_polarity_subjectivity'])['Comments'])
nps_worse=' '.join(input_survey[input_survey['nps_score'] < 3 ].nsmallest(n=50, columns=['Comments_spell_checked_txtblob_polarity','Comments_spell_checked_txtblob_polarity_subjectivity'])['Comments'])
#destribute that text blurb to people about the issues occuring

In [24]:
sentiment_chart=input_survey.groupby('Sentiment').size()
fig = px.pie(values=sentiment_chart,names=sentiment_chart.index,title='Semantic Count Overall')
fig.show()

In [25]:
import plotly.graph_objects as go
import plotly.express as px
mean_nps_time=input_survey.groupby('ActivityCompleted').mean()
fig = go.Figure(
    [go.Scatter(x=mean_nps_time.index, y=mean_nps_time['nps_score'])],
    layout=go.Layout(
    title=go.layout.Title(text="Average Satisfaction Score")
    )  
)
fig.show()

C:\Users\Chris Dong\AppData\Local\Temp\ipykernel_2016\119478978.py:3: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [9]:
# subjectivity information how does it compare to the built